Здесь будет написан небольшой отчёт о моей работе над изучением алгоритмов **IQL** и **AWAC**, которую я выполнял на протяжении последних десяти дней.

Из всех предложенных статей мне изначально более всего понравилась статья про **IQL** (https://arxiv.org/pdf/2110.06169.pdf). Алгоритм на момент выхода с отрывом (как мне показалось из бенчмарков) являлся SOTA, что добавило интереса к его изучению (всегда интересно изучать крутые и сильные методы). Статья написана понятно и достаточно подробно; в ней явно описывается проблема, которую пытаются решить авторы и чётко объяснены все выкладки. В алгоритме мне была непонятна часть с экспектильной регрессией, так как раньше с ней я не работал. Пришлось почитать дополнительные материалы, что помогло в полной мере осознать метод, предлагаемый авторами. 

Также хочу отметить большую и удобную таблицу с бенчмарками в этой статье, по ней и её описанию от авторов я выбрал датасеты, на которых было бы интересно наблюдать работу алгоритмов. А именно **hopper-medium-replay-v2** (эта среда, судя по бенчмаркам, является достаточно сложной для AWAC, но IQL справляется с ней хорошо. Мне захотелось подтвердить это, а также пронаблюдать процессы обучения обоих алгоритмов) и **halfcheetah-medium-replay-v2** (на этой среде все алгоритмы достигают примерно одинаковых результатов, что тоже показалось мне интересным). 

По описанию бенчмарков от авторов статьи я понял, что существуют два принципиально различных типа сред: 1-й содержит много траекторий близких к оптимальной (*locomotion*-среды), а 2-й содержит очень мало или вообще не содержит таковых (*antmaze*-среды). Многие алгоритмы (например "одношаговый" RL) обучаются хорошо на 1-м типе, но крайне плохо справляются со 2-м. Соответственно также интересно было бы протестировать алгоритмы и на 2-м типе сред, что я сделать к сожалению не успел, т.к. они имеют слегка отличающийся интерфейс, требующий отдельной реализации в коде.

Вторым алгоритмом для исследования я выбрал **AWAC** (https://arxiv.org/pdf/2006.09359.pdf). Решение взять его было продиктовано: а) относительной простотой в реализации(у меня оставалось 2 дня на его реализацию и обучение), б) схожей с уже реализованным IQL частью алгоритма (advantage-weighted регрессия для восстановления политики из неявной аппроксимации). Правда статья мне понравилась гораздо меньше, в начале льют очень много воды про то, какие миллионы проблем имеют другие offline to online rl подходы, и о том, про что вообще статья, читатель узнаёт примерно в её середине.

Про интерпретацию графиков (которые вообще говоря являются основными бенчмарками в моей работе): 
- Loss-графики: policy-loss, value-loss, q-loss для IQL и policy-loss, q-loss для AWAC - это просто показатели того, как сходятся нейросети, участвующие в обучении алгоритма, полезны для наблюдения момента, с которого начинается явное схождение политики, v-функции или q-функции к оптимальным значениям.
- Mean reward: основная метрика алгоритмов, т.к. вся наша цель в том, чтобы награда была большая.
- Initial state V: не такая очевидная метрика, показывает насколько агент в начале своей игры оценивает потенциальную награду, которую сможет получить. Так как большинство методов, стремящихся так или иначе максимизировать награду, склонны себя переоценивать, ухудшая этим стабильность своего обучения, я считаю эту метрику полезной и важной.

Краткое резюме по результатам обучения на **hopper-medium-replay-v2**:
- AWAC:
    - С дефолтными параметрами (beta=1) получил результат лучше, чем авторский.
    - С более строгой максимизацией награды (beta=3) результат был улучшен ещё.
    - Смягчение значения beta - не проверялось.
- IQL (имеет основные параметры tau и beta, я проверял только tau):
    - С дефолтными параметрами (tau=0.7) график средней награды находился в пределах чуть ниже авторских результатов.
    - С более жестким экспектилем (tau=0.9) ревард заметно упал. Связываю я это с тем, что метод не смог нормально сойтись из-за слишком строгого ограничения. Возможно ему потребовалось бы больше времени.
    - С более мягким экспектилем (tau=0.6) ревард также был ниже дефолтного, т.к. скорее всего поведение, близкое к описанному в датасете, оптимальным не является, а соответственно получает меньшие награды.

- Дополнительно: сходиться к оптимальным значениям политики в обоих алгоритмах начинали примерно в одинаковое время, общее время работы двух алгоритмов также идентично. Замечен факт того, что Initial-V тем больше, чем строже экспектиль мы выбрали для IQL, что логично (т.к. пытаемся аппроксимировать больший экспектиль). В сравнении двух алгоритмов видно, что AWAC более склонен к переоценке себя в начальном состоянии, что может быть связано с ошибками в обучении на действиях, которые не встречаются в датасете.



Проблемы с которыми я столкнулся при работе:
 - Очень проблемная установка d4rl на wsl2 в Windows. Половину действий пришлось делать руками и через костыли.
 - Отсутствие нормальных вычислительных мощностей: в наличии у меня был слабый ноутбук без видеокарты (хотя нейросети в поставленных задачах и не являлись bottleneck'ом).
 - Малое количество времени: я не успел провести эксперименты на halfcheetah-medium-replay-v2 и на *antmaze*-средах, также не успел перебрать оба параметра для IQL и в целом перебрать параметры по сетке нормального масштаба. Также хотел сам реализовать эпсилон-жадную политику, кроме описанной в авторских реализациях гауссовской. Было бы интересно посмотреть, как справляется выбор полностью случайного действия с exploration'ом на предложенных средах.
 - Наибольшее разочарование вызывает то, что я не успел обозреть online-часть обучения методов, так как, например, IQL должен получать значительно лучшие результаты именно в онлайне, так как не является консервативным методом, и не стремится быть близким к политике, задаваемой датасетом.
 - Также было бы неплохо разобраться с видео-репрезентацией действий агента, на что опять же не было времени.
 - Последняя проблема: я так и не смог понять, почему, несмотря на усреднение награды агента за 10 игр, график средней награды всё равно очень сильно "скачет". Возможно это происходит из-за ручного выставления фиксированного seed, но я не уверен в этом, и времени на проверку у меня уже не осталось.

 (с) Автор работы - Максим Скурихин.

 P.S. По возникшим вопросам в ходе проверки этого исследования я с удовольствием пообщаюсь с проверяющими лично - telegram: @ViV_99